import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

fg = pd.read_csv("field_goal_attempts.csv")
players = pd.read_csv("kickers.csv")

fg = fg[(fg['season_type'] == 'Reg') & (fg['week'] <= 6)]
fg = fg[fg['field_goal_result'].isin(['Made', 'Missed'])]
fg['make'] = fg['field_goal_result'].map({'Made': 1, 'Missed': 0})

fg.head()

model = LogisticRegression()
model.fit(fg[['attempt_yards']], fg['make'])
fg['expected_make_prob'] = model.predict_proba(fg[['attempt_yards']])[:, 1]
fg['fgoe'] = fg['make'] - fg['expected_make_prob']

kicker_stats = fg.groupby('player_id').agg(
    attempts=('make', 'count'),
    fgoe_total=('fgoe', 'sum'),
    fgoe_per_attempt=('fgoe', 'mean')
).reset_index()

kicker_stats = kicker_stats[kicker_stats['attempts'] >= 10].copy()
kicker_stats['rating'] = kicker_stats['fgoe_total']
kicker_stats['rank'] = kicker_stats['rating'].rank(method='min', ascending=False).astype(int)

leaderboard = kicker_stats.merge(players[['player_id', 'player_name']], on='player_id')
leaderboard = leaderboard[['player_id', 'player_name', 'rating', 'rank']].sort_values(by='rank')
leaderboard.head()

leaderboard.to_csv("leaderboard.csv", index=False)